In [7]:
%load_ext autoreload
%autoreload 2
from MCTS import MCTS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import time
from IPython.display import clear_output
import os
from stockfish import Stockfish


#path = os.getcwd()+"\stockfish_15.1_win_x64_avx2\stockfish-windows-2022-x86-64-avx2.exe"
#stockfish = Stockfish(path=path)
#stockfish.set_elo_rating(1150)

In [14]:
import chess
import random
board = chess.Board()
time_limit = 15

for i in range(1):
    move = random.choice(list(board.legal_moves))
    #board.push(move)

search1 = MCTS(board, time_limit=30, num_simulations=100)
search2 = MCTS(board, time_limit=30, num_simulations=100, heap_mark=True)

for i in range(50):
    """
    stockfish.set_fen_position(board.fen())
    board.push_san(stockfish.get_best_move())
    clear_output(wait=True)
    display(board)
    time.sleep(0.5)
    """
    result = None
    if board.turn:
        search1.search()
        result = search1.best_move()
    else:
        #result = random.choice(list(board.legal_moves))
        search2.search()
        result = search2.best_move()
    if result == None:
        break
    board.push(result)
    clear_output(wait=True)
    display(board)
    time.sleep(3)

KeyboardInterrupt: 